<a href="https://colab.research.google.com/github/Tulipraaj/Chatbot-using-RNN/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary Libraries

In [1]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

# Preprocessing Intents

In [2]:
with open('Intent.json') as f:
    intents = json.load(f)

In [3]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [4]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []

    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])

    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [5]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

    tokenizer.fit_on_texts(input_list)

    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')

    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [6]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])

    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [7]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


# Build Model

In [8]:
# hyperparameters
epochs=100
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [9]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirection  (None, 256)               656384    
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758166 (2.89 MB)
Trainable params: 758166 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [10]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/100
5/5 [==============================] - 6s 53ms/step - loss: 3.0908 - accuracy: 0.0420
Epoch 2/100
5/5 [==============================] - 0s 52ms/step - loss: 3.0667 - accuracy: 0.1119
Epoch 3/100
5/5 [==============================] - 0s 56ms/step - loss: 3.0195 - accuracy: 0.1538
Epoch 4/100
5/5 [==============================] - 0s 53ms/step - loss: 2.9594 - accuracy: 0.2238
Epoch 5/100
5/5 [==============================] - 0s 53ms/step - loss: 2.9028 - accuracy: 0.2308
Epoch 6/100
5/5 [==============================] - 0s 49ms/step - loss: 2.8052 - accuracy: 0.3007
Epoch 7/100
5/5 [==============================] - 0s 55ms/step - loss: 2.6473 - accuracy: 0.3566
Epoch 8/100
5/5 [==============================] - 0s 49ms/step - loss: 2.3881 - accuracy: 0.3916
Epoch 9/100
5/5 [==============================] - 0s 51ms/step - loss: 2.1482 - accuracy: 0.3776
Epoch 10/100
5/5 [==============================] - 0s 50ms/step - loss: 1.9014 - accuracy: 0.4965
Epoch 11/100
5/5 [=

# Initialize the Chatbot

In [14]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))

    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)

    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {}'.format(res))
    print()

Note: Enter 'quit' to break the loop.
You: hello
Bot: Hi human, please tell me your Bot user

You: tell me a joke
Bot: Two cows standing in a field. One turns to the other and says 'Moo!' The other one says 'Damn, I was just about to say that!'.

You: that was nice
Bot: OK

You: tell me a gossip
Bot: Cathy said she send him a mail where he wrote that he do not want to be his friend anymore.

You: thats bad
Bot: One second

You: quit
